# Amount of river Bad input

In [1]:
import sys
sys.path.insert(0, '../../_libs/')
import NEMO_tools as Nts
from Mat2Py import datenum2datetime, datetime2datenum_single

In [2]:
import glob
import scipy.io
import datetime
import numpy as np
#import seaborn as sb
import netCDF4 as nc
import NEMO_tools as Nts
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from NEMO_tools import reporj_NEMOgrid
from dateutil.relativedelta import relativedelta
% matplotlib inline

In [3]:
def seasonal_decomp3d(data, method=0):
    '''
    =======================================================================
    Remove the seasonal cycle from 1D data
                            ----- created on 2015/06/15, Yingkai (Kyle) Sha
    -----------------------------------------------------------------------
        data = seasonal_decomp(...)
    -----------------------------------------------------------------------
    Input:
            data: Time should be the first dim.
            method: removal done by anomaly (=0) or normalize (=1)
    ======================================================================= 
    '''
    data2 = np.empty(data.shape)
    for mon in range(12):
        temp_data = np.nanmean(data[mon:len(data):12, :, :], 0)
        if method == 0:
            data2[mon:len(data):12, :, :] = data[mon:len(data):12, :, :]-temp_data
    return data2

In [4]:
coordinate_name=glob.glob('/ocean/yingkai/GEOTRACES/NEMO-CODE/NEMOGCM/CONFIG/ANHA4_OFF/EXP00/mesh_zgr.nc')
coordinate_obj=nc.Dataset(coordinate_name[0])
nav_lon=coordinate_obj.variables['nav_lon'][:]
nav_lat=coordinate_obj.variables['nav_lat'][:]
e1t = coordinate_obj.variables['e1t'][:]
e2t = coordinate_obj.variables['e2t'][:]
e3t = coordinate_obj.variables['e3t_0'][:]

In [5]:
from dateutil.relativedelta import relativedelta
base = datetime.datetime(2002, 1, 1)
date_list = [base + relativedelta(months=x) for x in range(144)]

In [6]:
MAT = scipy.io.loadmat('../../_data/Exchange/Ba_ANHA4_SRC.mat')
Ba_ANHA4 = MAT['Ba_ANHA4'][:]
Ba_ANHA4 = np.transpose(Ba_ANHA4, [0, 2, 1])

In [7]:
rf_name = sorted(glob.glob('/ocean/yingkai/Repository/CCAR Modeling Team/analysis-Kyle/notebooks/_data/NEMO/ANHA4/runoff/*.nc')[2:])
rf_name = rf_name[44:-1]
NUM = len(rf_name)

In [8]:
rf_name

['/ocean/yingkai/Repository/CCAR Modeling Team/analysis-Kyle/notebooks/_data/NEMO/ANHA4/runoff/ANHA4_runoff_monthly_combined_Dai_Trenberth_Bamber_y2002.nc',
 '/ocean/yingkai/Repository/CCAR Modeling Team/analysis-Kyle/notebooks/_data/NEMO/ANHA4/runoff/ANHA4_runoff_monthly_combined_Dai_Trenberth_Bamber_y2003.nc',
 '/ocean/yingkai/Repository/CCAR Modeling Team/analysis-Kyle/notebooks/_data/NEMO/ANHA4/runoff/ANHA4_runoff_monthly_combined_Dai_Trenberth_Bamber_y2004.nc',
 '/ocean/yingkai/Repository/CCAR Modeling Team/analysis-Kyle/notebooks/_data/NEMO/ANHA4/runoff/ANHA4_runoff_monthly_combined_Dai_Trenberth_Bamber_y2005.nc',
 '/ocean/yingkai/Repository/CCAR Modeling Team/analysis-Kyle/notebooks/_data/NEMO/ANHA4/runoff/ANHA4_runoff_monthly_combined_Dai_Trenberth_Bamber_y2006.nc',
 '/ocean/yingkai/Repository/CCAR Modeling Team/analysis-Kyle/notebooks/_data/NEMO/ANHA4/runoff/ANHA4_runoff_monthly_combined_Dai_Trenberth_Bamber_y2007.nc',
 '/ocean/yingkai/Repository/CCAR Modeling Team/analysis-Ky

$$
\mathrm{Dissolved\ Barium\ river\ input\ (mol/s)} = \frac{1}{\rho_0}\mathrm{runoff}\Delta x\Delta y\cdot Ba_d\cdot 10^{-6}
$$
$$
\mathrm{Dissolved\ Barium\ residence\ time\ (s)} = \frac{Ba_d(d>50m)}{\mathrm{Dissolved\ Barium\ river\ input}+\mathrm{^*Pacific\ \&\ Atlantic\ input}}
$$

In [9]:
Ba_input = np.zeros([144, 800, 544])
rf_input = np.zeros([144, 800, 544])
for year in range(NUM):
    #print(year)
    nc_obj = nc.Dataset(rf_name[year])
    temp_rf = nc_obj.variables['runoff'][:]*e1t[0, :, :]*e2t[0, :, :]*1e-3   # rf*x*y*rho-1 kg/m2/s --> m3/s
    temp_rf[temp_rf==0]=np.nan
    for mon in range(12):
        #print(mon)
        Ba_input[12*year+mon, :, :] = temp_rf[mon, :, :]*Ba_ANHA4[mon, :, :]*1e-6 # mol/s
        rf_input[12*year+mon, :, :] = temp_rf[mon, :, :] # m3/s

In [10]:
rf_map=glob.glob('../../_data/Samples/STN-30p/runoff_id_modified.mat')
rf_obj=scipy.io.loadmat(rf_map[0])
rf_id=rf_obj['rf_id']
rf_lon=rf_obj['lon']
rf_lat=rf_obj['lat']
rf_id_interp=Nts.reporj_NEMOgrid(rf_lon, rf_lat, rf_id, nav_lon, nav_lat, method='nearest')

In [11]:
Ba_total    = np.nansum(np.nansum(Ba_input[:, :, :],  1), 1)
Ba_CAA = np.nansum(Ba_input[:, rf_id_interp==2], 1)
Ba_Barents  = np.nansum(Ba_input[:, rf_id_interp==4],  1)
Ba_Beaufort = np.nansum(Ba_input[:, rf_id_interp==5],  1)
Ba_Chukchi  = np.nansum(Ba_input[:, rf_id_interp==7],  1)
Ba_ESib     = np.nansum(Ba_input[:, rf_id_interp==8],  1)
Ba_Kara     = np.nansum(Ba_input[:, rf_id_interp==12], 1)
Ba_Laptev   = np.nansum(Ba_input[:, rf_id_interp==13], 1)
#rf_total    = np.nansum(np.nansum(rf_input[:, :, :],  1), 1)
#rf_Beaufort = np.nansum(rf_input[:, rf_id_interp==5],  1)
#rf_ESib     = np.nansum(rf_input[:, rf_id_interp==8],  1)
#rf_Kara     = np.nansum(rf_input[:, rf_id_interp==12], 1)
#rf_Laptev   = np.nansum(rf_input[:, rf_id_interp==13], 1)

In [21]:
#save_var = {'Ba_total': Ba_total, 'Ba_CAA': Ba_CAA, 'Ba_Barents': Ba_Barents, 'Ba_Beaufort': Ba_Beaufort,
#           'Ba_Chukchi': Ba_Chukchi, 'Ba_ESib': Ba_ESib, 'Ba_Kara': Ba_Kara, 'Ba_Laptev': Ba_Laptev}
#scipy.io.savemat('../../_data/MAT files/Ba_river_input.mat', mdict=save_var)

# Annual input in Eurasian rivers

In [20]:
MAT=scipy.io.loadmat('../../_data/MAT files/Processed samples/River_class.mat')
lon = MAT['lon_f']
lat = MAT['lat_f']
Class = MAT['River_class_f']
Class[Class==1]=0;
Class[Class==22]=1;

In [21]:
coordinate_name=glob.glob('/ocean/yingkai/GEOTRACES/NEMO-CODE/NEMOGCM/CONFIG/ANHA4_OFF/EXP00/mesh_zgr.nc')
coordinate_obj=nc.Dataset(coordinate_name[0])
nav_lon=coordinate_obj.variables['nav_lon'][:]
nav_lat=coordinate_obj.variables['nav_lat'][:]

In [24]:
Class_interp = reporj_NEMOgrid(lon, lat, Class, nav_lon, nav_lat, method='nearest')
Class_range = reporj_NEMOgrid(lon, lat, Class, nav_lon, nav_lat, method='linear')
Class_interp[np.isnan(Class_range)] = np.nan

In [25]:
d2m = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
s2m = 60.0*60*24*d2m
s2m_all = np.hstack([s2m, s2m, s2m, s2m, s2m, s2m, s2m, s2m, s2m, s2m, s2m, s2m])

In [26]:
def seasonal_cycle_1d(t):
    t_stack = np.reshape(t, [12, 12])
    return np.mean(t_stack, 0)

In [22]:
ANHA4_lon.shape

(544, 800)

In [23]:
Ba_input.shape

(144, 800, 544)

In [31]:
Ba_Lena=seasonal_cycle_1d(np.nansum(Ba_input[:, Class_interp==12],  1))*s2m*1e-8; 
np.sum(Ba_Lena)

0.61460616086130504

In [32]:
Ba_Lena=seasonal_cycle_1d(np.nansum(Ba_input[:, Class_interp==18],  1))*s2m*1e-8; 
np.sum(Ba_Lena)

0.7628557988693786

In [42]:
Ba_Lena=seasonal_cycle_1d(np.nansum(Ba_input[:, Class_interp==17],  1))*s2m*1e-8; 
np.sum(Ba_Lena)

0.42803572235751952